# SSIM

In [ ]:
from skimage.metrics import structural_similarity as ssim
import cv2

# Load the images in grayscale
image1 = cv2.imread('image1.jpg', cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread('image2.jpg', cv2.IMREAD_GRAYSCALE)

# Compute SSIM between the two images
ssim_index, ssim_map = ssim(image1, image2, full=True)

print("SSIM:", ssim_index)


In [ ]:
from skimage.metrics import structural_similarity as ssim
import numpy as np

# Example feature maps
feature_map1 = np.random.rand(64, 64)  # Example feature map
feature_map2 = np.random.rand(64, 64)  # Another example feature map

# Compute SSIM
ssim_index, ssim_map = ssim(feature_map1, feature_map2, full=True)

print("SSIM between feature maps:", ssim_index)


# Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Example feature vectors
feature_vector1 = np.random.rand(128)  # 128-dimensional feature vector
feature_vector2 = np.random.rand(128)

# Calculate cosine similarity
cosine_sim = cosine_similarity([feature_vector1], [feature_vector2])

print("Cosine Similarity:", cosine_sim[0][0])


# ORB

In [ ]:
import cv2

def orb_feature_compare(imageA, imageB):
    # Convert to grayscale
    imageA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    imageB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

    # Initialize ORB detector
    orb = cv2.ORB_create()

    # Find the keypoints and descriptors with ORB
    kpA, desA = orb.detectAndCompute(imageA, None)
    kpB, desB = orb.detectAndCompute(imageB, None)

    # Match descriptors
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(desA, desB)
    
    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)
    
    return len(matches), matches

# Load images
imageA = cv2.imread('image1.webp')
imageB = cv2.imread('image2.webp')

# Compare features using ORB
matches_count, matches = orb_feature_compare(imageA, imageB)
print(f"Number of Matches: {matches_count}")

# Grad-CAM

In [ ]:
import torchvision
import matplotlib.pyplot as plt


def prep_output(img_tensor):
    invTrans = torchvision.transforms.Compose([ torchvision.transforms.Normalize(mean = [ 0., 0., 0. ],
                                                    std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                            torchvision.transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                    std = [ 1., 1., 1. ]),
                            ])
    out = invTrans(img_tensor)
    out = out.cpu().detach().numpy().transpose(1, 2, 0)

    return out

def plot_maps(img1, img2,vmin=0.3,vmax=0.7, mix_val=2):
    f = plt.figure(figsize=(10,30))
    plt.subplot(1,3,1)
    plt.imshow(img1,vmin=vmin, vmax=vmax, cmap="gray")
    plt.axis("off")
    plt.subplot(1,3,2)
    plt.imshow(img2, cmap="gray")
    plt.axis("off")
    plt.subplot(1,3,3)
    plt.imshow(img1*mix_val+img2/mix_val, cmap="gray" )
    plt.axis("off")

def norm_flat_image(img):
    grads_norm = prep_output(img)
    grads_norm = grads_norm[:,:,0]+ grads_norm[:,:,1]+ grads_norm[:,:,2]
    grads_norm = (grads_norm - np.min(grads_norm))/ (np.max(grads_norm)- np.min(grads_norm))
    return grads_norm


probabilities = torch.nn.functional.softmax(imgs_out[0], dim=0)
top5_prob, top5_catid = torch.topk(probabilities, 5)
imgs_out[0, top5_catid[0]].backward()
norm_img = norm_flat_image(imgs[0])
norm_grad = norm_flat_image(imgs[0].grad)
plot_maps(norm_grad, norm_img)

# Main

In [1]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
import json
import argparse

from torchvision.models import wide_resnet50_2
from models.resnet import ResNet18
from torchvision.transforms import transforms
from dataset_loader import noise_loader, load_cifar10, load_svhn, load_cifar100, load_imagenet, load_mvtec_ad, load_visa

def loading_datasets(args, data_path, imagenet_path, transform):
    if args.dataset == 'cifar10':
        args.num_classes = 10
        train_loader, test_loader = load_cifar10(data_path,
                                            transforms=transform,
                                            batch_size=args.batch_size,
                                            one_class_idx=args.one_class_idx,
                                            seed=args.seed)
    elif args.dataset == 'svhn':
        args.num_classes = 10
        train_loader, test_loader = load_svhn(data_path, 
                                            transforms=transform,
                                            batch_size=args.batch_size,
                                            one_class_idx=args.one_class_idx,
                                            seed=args.seed)
    elif args.dataset == 'cifar100':
        args.num_classes = 20
        train_loader, test_loader = load_cifar100(data_path, 
                                                transforms=transform,
                                                batch_size=args.batch_size,
                                                one_class_idx=args.one_class_idx,
                                                seed=args.seed)
    elif args.dataset == 'imagenet30':
        args.num_classes = 30
        train_loader, test_loader = load_imagenet(imagenet_path, 
                                                transforms=transform,
                                                batch_size=args.batch_size,
                                                one_class_idx=args.one_class_idx,
                                                seed=args.seed)
    elif args.dataset == 'mvtec_ad':
        args.num_classes = 15
        train_loader, test_loader = load_mvtec_ad(data_path, 
                                                transforms=transform,
                                                resize=args.img_size,
                                                batch_size=args.batch_size,
                                                one_class_idx=args.one_class_idx,
                                                seed=args.seed)

    elif args.dataset == 'visa':
        args.num_classes = 12
        train_loader, test_loader = load_visa(data_path, 
                                            transforms=transform,
                                            resize=args.img_size,
                                            batch_size=args.batch_size,
                                            one_class_idx=args.one_class_idx,
                                            seed=args.seed)
    
    print("Start Loading noises")
    train_positives_loader, train_negetives_loader,\
    test_positives_loader, test_negetives_loader = noise_loader(
                    args,
                    transform=transform,
                    batch_size=args.batch_size,
                    one_class_idx=args.one_class_idx,
                    resize=args.img_size,
                    dataset=args.dataset,
                    preprocessing=args.preprocessing, 
                    k_pairs=args.k_pairs,
                    seed=args.seed)
    print("Loading noises finished!")

    return train_loader, test_loader, train_positives_loader, train_negetives_loader, test_positives_loader, test_negetives_loader


def parsing():
    parser = argparse.ArgumentParser(description='',
                                    formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--dataset', default='cifar10', type=str, help='cifar10-cifar100-svhn')
    parser.add_argument('--one_class_idx', default=None, type=int, help='select one class index')
    parser.add_argument('--config', default=None, help='Config file for reading paths')
    parser.add_argument('--batch_size', '-b', type=int, default=1, help='Batch size.')
    parser.add_argument('--seed', type=int, default=1, help='1')
    parser.add_argument('--img_size', default=32, type=int, help='image size selection')
    parser.add_argument('--preprocessing', default='clip', type=str, help='which preprocessing use for noise order')
    parser.add_argument('--k_pairs', default=1, type=int, help='Selecting multiple pairs for contrastive loss')
    parser.add_argument('--device', type=str, default="cuda", help='cuda or cpu.')
    args = parser.parse_args()

    return args





import sys
sys.argv = ["", "--one_class_idx", "0", "--dataset", "cifar10", "--batch_size", "1"]

with open('config.json', 'r') as config_file:
    config = json.load(config_file)

root_path = config['root_path']
data_path = config['data_path']
imagenet_path = config['imagenet_path']
args = parsing()
args.config = config
    

## Clip

In [ ]:
from clip import clip
model, transform = clip.load("ViT-L/14", device=args.device)

## Resnet-50 Wide

In [2]:
model = wide_resnet50_2("IMAGENET1K_V2")
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Resize(256),
                                            torchvision.transforms.CenterCrop(224)])

# Recursive function to register hooks for all layers, including nested ones
def register_hooks(model, feature_maps):
    hook_handles = []

    def hook_fn(module, input, output, layer_name):
        feature_maps[layer_name] = output

    def recursive_hook_register(module, parent_name=""):
        for name, sub_module in module.named_children():
            full_name = f"{parent_name}.{name}" if parent_name else name
            handle = sub_module.register_forward_hook(
                lambda module, input, output, name=full_name: hook_fn(module, input, output, name)
            )
            hook_handles.append(handle)
            # Recursively register hooks for nested layers
            recursive_hook_register(sub_module, full_name)

    recursive_hook_register(model)
    return hook_handles


# Forward pass and capture feature maps
def get_all_feature_maps(model, img_tensor):
    feature_maps = {}
    hook_handles = register_hooks(model, feature_maps)

    # Forward pass
    model(img_tensor)

    # Remove hooks
    for handle in hook_handles:
        handle.remove()

    return feature_maps

C:\Users\Mohammad\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


## Run

In [3]:
train_loader, test_loader, train_positives_loader, train_negetives_loader,\
    test_positives_loader, test_negetives_loader = loading_datasets(args, data_path, imagenet_path, transform=transform)


loading cifar10
Files already downloaded and verified
Files already downloaded and verified
Start Loading noises
Creating noises loader
Selecting flip as positive pair for class 0
Selecting glass_blur as negetive pair for class 0
Loading noises finished!


In [ ]:
imgs_out = []
p_imgs_out = []
n_imgs_out = []
model = model.to(args.device)
model.eval()

iter = 0
for (normal, p_data, n_data) in zip(train_loader, train_positives_loader[0], train_negetives_loader[0]):
    imgs, labels = normal
    p_imgs, p_label = p_data
    n_imgs, n_label = n_data
    assert torch.equal(torch.squeeze(labels), torch.squeeze(p_label)), f"The labels of positives images do not match to noraml images, {torch.squeeze(labels)}, {torch.squeeze(p_label)}"
    assert torch.equal(torch.squeeze(labels), torch.squeeze(n_label)), f"The labels of negatives images do not match to noraml images, {torch.squeeze(labels)}, {torch.squeeze(n_label)}"
    imgs, labels = imgs.to(args.device), labels.to(args.device)
    p_imgs = p_imgs.to(args.device)
    n_imgs = n_imgs.to(args.device)

    imgs.requires_grad = True
    p_imgs.requires_grad = True
    n_imgs.requires_grad = True

    # imgs_out = model.encode_image(imgs)
    # p_imgs_out = model.encode_image(p_imgs)
    # n_imgs_out = model.encode_image(n_imgs)

    # imgs_out.append(model.encode_image(imgs))
    # p_imgs_out.append(model.encode_image(p_imgs))
    # n_imgs_out.append(model.encode_image(n_imgs))

    # imgs_out = model(imgs)
    # p_imgs_out = model(p_imgs)
    # n_imgs_out = model(n_imgs)

    imgs_out.append(get_all_feature_maps(model, imgs)['avgpool'].view(-1, 2048).cpu().detach().numpy()[0])
    p_imgs_out.append(get_all_feature_maps(model, p_imgs)['avgpool'].view(-1, 2048).cpu().detach().numpy()[0])
    n_imgs_out.append(get_all_feature_maps(model, n_imgs)['avgpool'].view(-1, 2048).cpu().detach().numpy()[0])
    if iter > 128:
        break
    iter += 1

In [5]:
x = np.concatenate((np.array(imgs_out), np.array(p_imgs_out)), axis=0)
# x = np.concatenate((imgs_out[0].cpu().detach().numpy(), p_imgs_out.cpu().detach().numpy()), axis=0)
y = np.concatenate((np.ones(len(imgs_out)), np.zeros(len(p_imgs_out))), axis=0)

In [6]:
x.shape, y.shape

((260, 2048), (260,))

In [7]:
from sklearn.manifold import TSNE
import pandas as pd
import plotly.express as px 

In [8]:
tsne = TSNE(n_components=2, verbose=1, random_state=123, learning_rate='auto')
z = tsne.fit_transform(x) 
df = pd.DataFrame()
df["y"] = y
df["component_1"] = z[:,0]
df["component_2"] = z[:,1]
label_str = []
for l in df.y.tolist():
    if l == 1:
        label_str.append('0')
    else:
        label_str.append('1')

fig = px.scatter(data_frame=df, x="component_1", y="component_2", color=label_str,
           labels={
                     "component_1": "Component 1",
                     "component_2": "Component 2",
                 },
            # title="",
            width=1024, height=1024)

fig.update_layout(
    font=dict(
        size=22,  # Set the font size here
    )
)
fig.show()
# fig.write_image(original_path+prefix_path+config_path+"{}_vs_{}_tsne.png".format(class1, class2))

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 260 samples in 0.001s...
[t-SNE] Computed neighbors for 260 samples in 0.166s...
[t-SNE] Computed conditional probabilities for sample 260 / 260
[t-SNE] Mean sigma: 5.731060
[t-SNE] KL divergence after 250 iterations with early exaggeration: 54.045128
[t-SNE] KL divergence after 1000 iterations: 0.479017
